In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_excel("/kaggle/input/nbfc-housing/NBFC HOUSING FINANCE.xlsx")
df

In [ ]:
df.columns = ['Name', 'CMP', 'MKT_CAP' ,'PE', 'PB','ROA','ROE','Promoter_holdings','Ind PE',
               'Sales_growth_1','Profit_growth_1','FCF','Sales_growth_3','Profit_growth_3','EPS_growth_3','EPS','EPS_1']

In [ ]:
df

In [ ]:
box_a = df[df['PE'] > 22.65]
box_b = df[df['PE'] <= 22.65]

In [ ]:
box_b['Target_1'] = round(box_b['PE']*box_b['CMP'],2)

In [ ]:
box_b['Target_2'] = round((box_b['PE'] + box_b['FCF'])*box_b['CMP'],2)

In [ ]:
box_b = box_b.dropna()

In [ ]:
quantiled = box_b['ROE'].quantile([0.25,0.50,0.75])
quantiled

In [ ]:
score = []

for value in box_b['ROE']:
    if value <= quantiled.values[0]:
        score.append(int(1))
    elif quantiled.values[0] < value <= quantiled.values[1]:
        score.append(int(2))
    elif quantiled.values[1] < value <= quantiled.values[2]:
        score.append(int(3))
    elif value > quantiled.values[2]:
        score.append(int(4))

box_b['shareholder_value_score'] = score

In [ ]:
box_b

In [ ]:
quantiled = box_b['ROA'].quantile([0.25,0.50,0.75])
quantiled

In [ ]:
score = []

for value in box_b['ROA']:
    if value <= quantiled.values[0]:
        score.append(int(1))
    elif quantiled.values[0] < value <= quantiled.values[1]:
        score.append(int(2))
    elif quantiled.values[1] < value <= quantiled.values[2]:
        score.append(int(3))
    elif value > quantiled.values[2]:
        score.append(int(4))

box_b['asset_quality_score'] = score

In [ ]:
box_b['market_aperture(X)'] = round(box_b['PB'],0)

In [ ]:
box_b['shareholders_value_finance'] = round(box_b['PB']*box_b['CMP'],2)

# **GROWTH BASED TARGET**

In [ ]:
count_of_rate_hikes = 1 
N = 1
inflation_rate = 0.067 ## RBI DATA
sector_growth_rate = 0.12+0.04 ## sector growth rate + cost of fund  ## ICRA DATA
lambda_ = 0.10

box_b['sales1_grwth_target'] = round((((box_b['Sales_growth_1'])/100 + 1)**(N - count_of_rate_hikes))*(box_b['CMP']*(1+sector_growth_rate - (inflation_rate)*(lambda_))),2)
box_b

In [ ]:
count_of_rate_hikes = 2 
N = 3
inflation_rate = 0.067 ## RBI DATA
sector_growth_rate = 0.12+0.04 ## sector growth rate + cost of fund  ## ICRA DATA
lambda_ = 0.80

box_b['sales3_grwth_target'] = round((((box_b['Sales_growth_3'])/100 + 1)**(N - count_of_rate_hikes))*(box_b['CMP']*(1+sector_growth_rate - (inflation_rate)*(lambda_))),2)
box_b

In [ ]:
count_of_rate_hikes = 1 
N = 1
inflation_rate = 0.067 ## RBI DATA
sector_growth_rate = 0.12 ## sector growth rate + cost of fund  ## ICRA DATA
lambda_ = 0.10

box_b['profit1_grwth_target'] = round((((box_b['Profit_growth_1'])/100 + 1)**(N - count_of_rate_hikes))*(box_b['CMP']*(1+sector_growth_rate - (inflation_rate)*(lambda_))),2)
box_b

In [ ]:
count_of_rate_hikes = 2 
N = 3
inflation_rate = 0.067 ## RBI DATA
sector_growth_rate = 0.12 ## sector growth rate + cost of fund  ## ICRA DATA
lambda_ = 0.80

box_b['profit3_grwth_target'] = round((((box_b['Profit_growth_3'])/100 + 1)**(N - count_of_rate_hikes))*(box_b['CMP']*(1+sector_growth_rate - (inflation_rate)*(lambda_))),2)
box_b

# **WEIGHTED AVERAGE OF ALL TARGET VARIABLES**

**1) sales1_grwth_target : 19.8%**
**2) sales3_grwth_target : 19.5%**
**3) profit1_grwth_target : 30.2%**
**4) profit3_grwth_target : 30.2%**

In [ ]:
box_b['Final_Target'] =round(box_b['sales1_grwth_target']*(0.198) + box_b['sales3_grwth_target']*(0.195) + \
                                        box_b['profit1_grwth_target']*(0.302) + box_b['profit3_grwth_target']*(0.302) + box_b['shareholders_value_finance']*(0.0298) + box_b['Target_1']*(0.001) + box_b['Target_2']*(0.001),2)

In [ ]:
box_b

In [ ]:
box_b.columns

In [ ]:
box_b['expected_upflow%'] = round((box_b['Final_Target']-box_b['CMP'])*100/box_b['CMP'],2)
box_b

In [ ]:
series_a = box_b[((box_b['asset_quality_score']) > 1) & ((box_b['shareholder_value_score']) > 1) & ((box_b['market_aperture(X)'] <= 3))]
series_a

In [ ]:
series_a['Allocated_Amount'] = 100000

In [ ]:
series_a['Constant_allocation'] = series_a['Allocated_Amount']*series_a['expected_upflow%']/100

In [ ]:
series_a

In [ ]:
if sum(series_a['Constant_allocation']) > (series_a['Allocated_Amount'].values[0]):
    print("RAISE ERROR : 101")
    print("***********************")
    print("FUNDS INSUFFICIENT !!!!!!")
    print("MANUAL INTERVENTION REQUIRED........")

# **MANUALLY INTERVENED**

In [ ]:
## Decreasing the allocation amount for repco home finance scrip and srg housing finance scrip

In [ ]:
series_a['Constant_allocation'] = [20000.0,10000.0,17000.0,28000,25000]

In [ ]:
sum(series_a['Constant_allocation'])

In [ ]:
series_a

In [ ]:
series_a.to_excel("series_A_NBFC_HOUSING_Portfolio.xlsx",index=False)